# Map matching
We have cleaned our data, now we can start our study on the taxi GPS trajectories. Our first step is to perform **map-matching** on the GPS trajectory data to get the corrected GPS coordinates and the road ID sequence for each trajectory.

### What is map matching
Map matching is the process of aligning raw GPS data points with a known road network to determine the most likely path traveled. Since GPS readings can be imprecise due to various factors, map-matching algorithms help improve accuracy by snapping these points to the road network. Techniques vary from geometric approaches to more advanced probabilistic models like Hidden Markov Models (HMM), which account for both the GPS noise and the logical connectivity of roads.

### Map matching with Valhalla's Meili
The map matching tool we'll be using is **Valhalla's Meili**. [Valhalla](https://valhalla.github.io/valhalla/) is an open source routing engine and accompanying libraries for use with OpenStreetMap data. [Meili](https://valhalla.github.io/valhalla/api/map-matching/api-reference/) is Valhalla's map matching service that aligns GPS traces to known roads or paths by comparing raw location data to OpenStreetMap's detailed road network. It can handle large-scale spatial data and is optimized for high-performance processing, making it ideal for applications like traffic analysis or location-based services.